In [2]:
!pip install webdriver-manager

In [3]:
!pip install selenium

In [26]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [55]:
# Chrome WebDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

# Chrome WebDriver 초기화 및 웹 페이지 열기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)
url = "https://www.hufs.ac.kr/hufs/11282/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGaHVmcyUyRjIxODElMkZhcnRjbExpc3QuZG8lM0Y%3D" #학사 홈페이지
driver.get(url)

In [56]:
# csv에 담아 출력할 DataFrame 생성
import pandas as pd
df_crawling = pd.DataFrame(columns = ['index', 'date', 'title', 'content'])

In [57]:
# index 수짐
# Selenium으로 페이지의 HTML 소스 가져오기
html = driver.page_source

# BeautifulSoup 객체 생성하여 HTML 파싱
soup = BeautifulSoup(html, 'html.parser')

# CSS 선택자로 인덱스 추출
elements = soup.select("div._fnctWrap table tbody tr:not(.notice) td.td-num")
index_texts = [element.get_text() for element in elements]

# 추출한 인덱스를 숫자로 변환
index = int(index_texts[0]) if index_texts else None

In [58]:
# 첫번째 공지 글 클릭
# CSS 선택자를 사용하여 공지태그가 아닌 첫 번째 글의 <a> 태그를 찾기
first_page = driver.find_element(By.CSS_SELECTOR, "div.scroll-table tbody tr:not(.notice) a")

# 클릭
first_page.click()

In [59]:
# 반복문 시작-----------------------------

In [60]:
while True:
    try:
        # 페이지의 소스 가져오기
        html_source = driver.page_source
        
        # BeautifulSoup 객체 생성하여 HTML 파싱, 'html.parser' 대신 다른 파서를 사용할 수도 있습니다.
        soup = BeautifulSoup(html_source, 'html.parser')
        
        # CSS 선택자를 사용하여 날짜, 제목, 내용 추출
        date = soup.select_one("div.view-info div.view-detail dl.write dd").get_text(strip=True) if soup.select_one("div.view-info div.view-detail dl.write dd") else ''
        title = soup.select_one("div.board-view-info div.view-info h2").get_text(strip=True) if soup.select_one("div.board-view-info div.view-info h2") else ''
        content = soup.select_one("div.view-con").get_text(strip=True) if soup.select_one("div.view-con") else ''
        
        # 잘 추출되었는지 확인
        print(f"Date: {date}")
        print(f"Title: {title}")
        print(f"Content: {content}")

        # 텍스트 정리(탭, 줄바꿈)
        #title = title.replace("\n", "").replace("\t", "")
        #content = content.replace("\n", "").replace("\t", "")
        
        # 텍스트 정리(탭)
        title = title.replace("\t", "").replace('\xa0', '')
        content = content.replace("\t", "").replace('\xa0', '')

        # 데이터프레임에 추가
        new_row = {'index': index, 'date': date, 'title': title, 'content': content}
        df_crawling.loc[len(df_crawling)] = new_row
        
        # 인덱스 갱신
        index -= 1

        # 이전글 클릭
        # CSS 선택자를 사용하여 이전글의 <a> 태그를 찾기
        previous_page = driver.find_element(By.CSS_SELECTOR, "main.contents div._fnctWrap div.view-navi dl:first-child dd a")
        
        # 클릭
        previous_page.click()
        time.sleep(2)
        
    except Exception as e:
        print(e)
        break

Date: 2024.03.26
Title: [공통]2024-여름계절학기 개설희망 교과목 수요조사 실시 공고
Content: 계절학기 개설희망 교과목 수요조사 실시 공고2024학년도 여름계절학기 개설희망 교과목에 대한 수요조사 실시를 아래와 같이 공고함-아     래 -1.수요조사 기간: 2024. 4. 2(화) 10:00 ~ 4. 4(목) 16:002.참여대상: 2024학년도 1학기 재학생(휴학생, 교류생은 참여 불가)3.참여방법가. 학교 홈페이지 → HOT LINK → “2024-여름계절학기 수요조사” 클릭 후 로그인나. 개설을 희망하는 교과목 신청다. 수요조사는계절학기 수강학점(최대 6학점) 한도 내에서 신청 가능※ 캠퍼스간 교차 수요조사 참여가 가능하며 학과(부)간 신청의 제한 없음(예-글로벌캠퍼스 학생도 서울캠퍼스의 교과목에 대해 수요조사 참여 가능하며서울캠퍼스 학생도 글로벌캠퍼스 교과목에 대해서 수요조사 참여 가능,또한 본인 전공 외에 타 학과의 전공에 대해서도 수요조사 참여 가능)※ 자세한 사항은 첨부된 [계절학기 개설희망 교과목 수요조사 참여 안내문] 확인4.유의사항가. 수요조사는 정규학기에 개설된 교과목을 대상(여름계절학기는 1학기 개설 교과목,겨울계절학기는 2학기 개설 교과목 대상)으로 하며 캠퍼스·학과(부)간 수요조사 참여의 제한은 없습니다.나. 본 수요조사는 강좌개설을 위한 참고자료로만 활용되며,실제 강좌 개설 여부는 각 학과(부) 사무실로 문의하시기 바랍니다.다. 실제 수강신청 시 해당 교과목의 수강제한 여부와 학생의 과거 수강 내역 등에 따라수강 신청이 제한될 수 있습니다.라. 정확한 수요조사를 위해 불필요한 신청은 자제해 주시기 바랍니다.2024.  3.  26.서울.글로벌캠퍼스 교무처장
Date: 2024.03.15
Title: [글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스)
Content: 2024-1학기 폐강(2차)공고(글로벌캠퍼스)2024-1학기글로벌캠퍼스 폐강(2차)강좌를 아래와 같이 공고합니다.폐강 강좌를 수강신청한

In [ ]:
# 반복문 종료---------------------

In [61]:
# Selenium WebDriver 종료
driver.quit()

In [63]:
# csv 파일 export
df_crawling.to_csv("C:/Users/admin/Desktop/noti_bachelor.csv", encoding="UTF-8", index=False)